# Random forest

Los bosques aleatorios se trata de un algoritmo predictivo el cual nos permite *hacer crecer* árboles, ya sean de regresión como de clasificación.
En el caso de la regresión, realizaremos un promedio de los árboles.
Mientras que en el caso de la clasificación se hará una votación y el resultado será el que dicte la mayoría.

Esto nos dice que tenemos dos métodos generales para el ensamblaje de los distintos algoritmos.

* Métodos de promedio: se crean varios modelos similares e independientes y se hace un promedio de las predicciones de cada modelo.
* Métodos de impulso: intentamos reducir el sesgo del estimador combinado, construyéndolo de forma secesiva de los estimadores base. Así conseguimos un modelo más robusto a partir de varios más débiles.

## Algunas ventajas de Random Forest

* No necesita validación cruzada, ya que utilzia el método de **bagging o bootstrap**. El método de **bagging** consiste en:
    * Con $n$ observaciones en el dataset de entrenamiento $T$ y $m$ variables, decidimos hacer crecer $S$ árboles en nuestro bosque. Cada uno será creado con un dataset de entrenamiento separado.
    * Las $n$ observaciones para cada dataset se toman aleatoriamente con reemplazo del dataset original.
    * Cada dataset puede tener observaciones duplicadas y algunas no aparecer nunca como entrenamiento.
    * Estos datasets se llaman muestras de *bootstrap*.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# seaborn style
sns.set()

In [2]:
# Importamos librarías específicas
from sklearn.ensemble import RandomForestRegressor

In [3]:
data = pd.read_csv('../../datasets/boston/Boston.csv')
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [4]:
colnames = data.columns.values.tolist()
predictors = colnames[:13]
target = colnames[13]
X = data[predictors]
y = data[target]

In [9]:
forest = RandomForestRegressor(n_jobs=2,
                               oob_score=True,
                               n_estimators=100)
forest.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [11]:
# añadimos al dataset la predicción
data['rforest_pred'] = forest.oob_prediction_
data[["rforest_pred", "medv"]].head(10)

,rforest_pred,medv
0,28.103333,24.0
1,22.203333,21.6
2,33.678571,34.7
3,34.674074,33.4
4,34.789286,36.2
5,25.225714,28.7
6,20.318182,22.9
7,17.679310,27.1
8,18.532432,16.5
9,19.615789,18.9


Aquí hemos obtenido el valor que ha predicho el algoritmo *Random Forest* con un número de 100 árboles, y el valor original.

Podemos comprobar el error cuadrático medio para comparar con el modelo de un único árbol.

In [13]:
data["rforest_error2"] = (data["rforest_pred"] - data["medv"])**2
sum(data["rforest_error2"]) / len(data)

11.111799395043866

Otro coeficiente que podemos consultar, y que tiene un significado similar al $r^{2}$

In [14]:
forest.oob_score_

0.8683741078371883